# Applied Machine Learning: Credit Card Approval Prediction

*By Daniel Deutsch and José Lucas Barretto*

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import seaborn as sns


In [2]:
# Matplotlib styles
plt.style.use('ggplot')
plt.rcParams.update({
    'figure.figsize': (15, 4),
    'axes.prop_cycle': plt.cycler(color=["#4C72B0", "#C44E52", "#55A868", "#8172B2", "#CCB974", "#64B5CD"]),
    'axes.facecolor': "#EAEAF2"
})

# random state
random_state=42

## Data Processing

At this staage the goal is to read both datasets and adapt its content for our classification models.

### Processing raw data

#### Credit records

In [3]:
# Reads the dataset
df_credit_record = pd.read_csv("./datasets/credit_record.csv")

# Renames columns
df_credit_record.rename(columns=str.lower, inplace=True)

# Numericalize status
df_credit_record.replace({'status': {'X': 0, 'C': 0, '0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6}}, inplace=True)

# Calculates the metric
df_credit_record = df_credit_record.groupby('id')['status'].mean().reset_index()

# Sets the label
df_credit_record['is_good'] = 1*(df_credit_record['status'] <= 1)

#### Application Records

In [4]:
# Reads the dataset
df_application_record = pd.read_csv("./datasets/application_record.csv")

# Renames columns
df_application_record.rename(columns=str.lower, inplace=True)

# numericalyze and rename education level
df_application_record.replace({
    'name_education_type': {
        'Higher education': 4,
        'Academic degree': 4,
        'Incomplete higher': 3,
        'Secondary / secondary special': 2,
        'Lower secondary': 1, 
    }
}, inplace=True)
df_application_record.rename(columns={'name_education_type': 'education_level'}, inplace=True)

# transform days employed into is_unemployed
df_application_record['is_unemployed'] = 1*(df_application_record['days_employed'] > 0)

# drop uninteresting columns - birthday, days employed
df_application_record.drop(columns=['days_employed', 'days_birth'], inplace=True)

#### Merging data

In [5]:
df = pd.merge(df_credit_record, df_application_record, how='inner', on='id')

del df_credit_record, df_application_record

### Train-test split

In [6]:
# define categorical, numerical, and other interesting vars
categorical = [
    'code_gender', 
    'flag_own_car', 
    'flag_own_realty', 
    'name_income_type',
    'name_family_status', 
    'name_housing_type', 
    'flag_mobil', 
    'flag_work_phone',
    'flag_phone', 
    'flag_email',
    'occupation_type'
]

numerical = [
    'cnt_children',
    'amt_income_total',
    'cnt_fam_members'
]

other = [
    'is_unemployed',
    'education_level'
]

# define X and y datasets
X = df[categorical + numerical + other]
y = df[['is_good']]

# split between train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

### Processing features

In [7]:
# initialize encoder for categorical vars
enc = OneHotEncoder(drop='first', sparse=False)

# fit transform encoder to train data
X_train_categorical = pd.DataFrame(enc.fit_transform(X_train[categorical]), index=X_train.index, columns=enc.get_feature_names_out())

# transform encoder to test data
X_test_categorical = pd.DataFrame(enc.transform(X_test[categorical]), index=X_test.index, columns=enc.get_feature_names_out())

In [8]:
# initialize scaler for numerical vars
scaler = StandardScaler()

# fit transform scaler to train data
X_train_numerical = pd.DataFrame(scaler.fit_transform(X_train[numerical]), index=X_train.index, columns=numerical)

# transform scaler to test data
X_test_numerical = pd.DataFrame(scaler.transform(X_test[numerical]), index=X_test.index, columns=numerical)

In [9]:
# concatenate numerical and categorical vars
X_train_normal = pd.concat((X_train_numerical, X_train_categorical, X_train[other]), axis=1)
X_test_normal = pd.concat((X_test_numerical, X_test_categorical, X_test[other]), axis=1)

## Feature selection

## Predictive Analysis